# McKinsey scrape

The following code scrapes the McKinsey site for articles, pages etc. about AI. The scraper collects text from pages retuned from searching "artificial intelligence on McKinsey.com: https://www.mckinsey.com/search?q=artificial+intelligence

### Libraries

Importing libraries...

In [2]:
import scrapy
import requests
from scrapy import Selector

import os
import sys
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

import time

### URL list

List of URL is extracted from searching for "artificial intelligence" on McKinsey.com: https://www.mckinsey.com/search?q=artificial+intelligence

In [5]:
start_url = 'https://www.mckinsey.com/search?q=artificial+intelligence' #main URL

Scraping article URLs

In [47]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options

chromeOptions = Options()
chromeOptions.add_argument("--kiosk")

driver = webdriver.Chrome(executable_path = 'C:\\bin\\chromedriver', options=chromeOptions)

driver.get(start_url)
time.sleep(2)
driver.find_element_by_css_selector('a.btn.cookie-btn.cc-dismiss').click()
time.sleep(2)

pageSel = Selector(text = driver.page_source)

urls = pageSel.css('div.search-results > div > div> div > a::attr(href)').extract()
titles = pageSel.css('a > h3::text').extract()

urldict = {}
urldict['urls'] = urls
urldict['titles'] = titles

coordinates = {'x': 0, 'y': 1600}
driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))

while True:
    try:
        time.sleep(2)
        driver.find_element_by_css_selector('div.search-results-footer.-show-next > ul > li > a.mck-arrow-right-icon').click()
        time.sleep(2)
        driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
        time.sleep(2)
        
        pageSel = Selector(text = driver.page_source)

        urls = pageSel.css('div.search-results > div > div> div > a.item-title-link::attr(href)').extract()
        titles = pageSel.css('a > h3::text').extract()

        urldict['urls'] = urldict['urls'] + urls
        urldict['titles'] = urldict['titles']+ titles
    except (NoSuchElementException, WebDriverException, ElementNotInteractableException) as e:
        print("No 'Next' button found")
        break

driver.quit()

No 'Next' button found


In [48]:
print(len(urldict['urls']),
     len(urldict['titles']))

912 909


In [51]:
oneurl = urldict['urls'][1]
oneurl

'https://www.mckinsey.com/featured-insights/artificial-intelligence/the-ethics-of-artificial-intelligence'

In [53]:
urllist = urldict['urls']
len(urllist)

912

In [54]:
with open('../data_raw/urls/mckinsey_urls.txt', 'w') as f:
    for url in urllist:
        f.write(url + "\n")

### Load URLs

In [62]:
mckinsey_urls = []
with open('../data_raw/urls/mckinsey_urls.txt', 'r') as f:
    for line in f:
        mckinsey_urls.append(line.strip())

print(len(mckinsey_urls),
      '\n',
     mckinsey_urls[len(mckinsey_urls)-1])

912 
 https://www.mckinsey.com/business-functions/organization/our-insights/the-irrational-side-of-change-management


In [63]:
urllist = mckinsey_urls

### Testing one page

In [115]:
page_url = urllist[120]
page_url

'https://www.mckinsey.com/business-functions/mckinsey-analytics/our-insights/ask-the-ai-experts-whats-driving-todays-progress-in-ai'

In [116]:
page_dict = dict()
main_dict = dict()

In [117]:
text_sel = 'div#divArticleBody > article.main-copy ::text'
title_sel = '//meta[contains(@property, "og:title")]/@content'

script_sel = '//script[contains(@type, "text/javascript")]/text()'

date_regex = r'(?<=\"OriginalPublicationDate\"\:\")\d{1,2}/\d{1,2}/\d{4}'
texttype_regex = r'(?<=\"ArticleType\"\:\")\w*?(?=\",)'
conttype_regex = r'(?<=\"ContentType\"\:\")\w*?(?=\",)'

page_html = requests.get(page_url).content
page_sel = Selector(text = page_html)

page_script = page_sel.xpath(script_sel).extract_first()

content_type = re.findall(conttype_regex, page_script)[0]

siteno = 1

if content_type != "Profile":

    page_date = re.findall(date_regex, page_script)[0]
    page_texttype = re.findall(texttype_regex, page_script)[0]

    page_id = 'site' + str(siteno)
    page_title = page_sel.xpath(title_sel).extract_first()
    page_dldate = date.today().strftime('%Y/%m/%d')
    
    page_text = page_sel.css(text_sel).extract()
    page_text = ' '.join(page_text)
    
    page_dict['url'] = page_url
    page_dict['title'] = page_title
    page_dict['article date'] = page_date
    page_dict['download date'] = page_dldate
    page_dict['text type'] = page_texttype
    page_dict['text'] = page_text

    main_dict[page_id] = page_dict
    

In [118]:
print(len(main_dict),
      main_dict['site1'])

1 {'url': 'https://www.mckinsey.com/business-functions/mckinsey-analytics/our-insights/ask-the-ai-experts-whats-driving-todays-progress-in-ai', 'title': 'Ask the AI experts: What&rsquo;s driving today&rsquo;s progress in AI?', 'article date': '7/7/2017', 'download date': '2020/02/18', 'text type': 'Video', 'text': '\r\n \r\n     \r\n         Downloadable Resources \r\n         Open interactive popup \r\n         \r\n                     \r\n                         \r\n                             Article (PDF-46KB) \r\n                         \r\n                     \r\n         \r\n     \r\n \r\n  \n Artificial-intelligence technology  has begun to hit its stride, springing from research labs into real business and consumer applications. But why now? Earlier this year at the  AI Frontiers conference  in Santa Clara, California, we sat down with AI experts from some of the world’s leading technology-first organizations to find out. An edited version of the experts’ remarks follows t

In [119]:
urllist_snip = urllist[:4]
urllist_snip

['https://www.mckinsey.com/featured-insights/artificial-intelligence',
 'https://www.mckinsey.com/featured-insights/artificial-intelligence/the-ethics-of-artificial-intelligence',
 'https://www.mckinsey.com/featured-insights/artificial-intelligence/ai-looks-north-bridging-canadas-corporate-artificial-intelligence-gap',
 'https://www.mckinsey.com/featured-insights/artificial-intelligence/tackling-bias-in-artificial-intelligence-and-in-humans']

In [120]:
page_dict = dict()
main_dict = dict()

In [121]:
len(main_dict)

0

## McKinsey spider

In [122]:
from scrapy.crawler import CrawlerProcess
from datetime import date

# Create class
class mckinsey_spider (scrapy.Spider):
    name = "mckinsey_spider"
    
    def start_requests(self):
        urls = urllist
        
        for url in urls:
            yield scrapy.Request(url = url, callback = self.parse)
            
    #Parsing
    def parse(self, response):
        print(response.url)
        
        page_dict = dict()
        siteno = len(main_dict) + 1
        
        page_url = response.url
        
        page_html = requests.get(page_url).content
        page_sel = Selector(text = page_html)
        
        text_sel = 'div#divArticleBody > article.main-copy ::text'
        title_sel = '//meta[contains(@property, "og:title")]/@content'

        script_sel = '//script[contains(@type, "text/javascript")]/text()'

        date_regex = r'(?<=\"OriginalPublicationDate\"\:\")\d{1,2}/\d{1,2}/\d{4}'
        texttype_regex = r'(?<=\"ArticleType\"\:\")\w*?(?=\",)'
        conttype_regex = r'(?<=\"ContentType\"\:\")\w*?(?=\",)'

        page_script = page_sel.xpath(script_sel).extract_first()

        content_type = re.findall(conttype_regex, page_script)[0]
        
        if content_type != "Profile":

            page_date = re.findall(date_regex, page_script)[0]
            page_texttype = re.findall(texttype_regex, page_script)[0]

            page_id = 'site' + str(siteno)
            page_title = page_sel.xpath(title_sel).extract_first()
            page_dldate = date.today().strftime('%Y/%m/%d')

            page_text = page_sel.css(text_sel).extract()
            page_text = ' '.join(page_text)

            page_dict['url'] = page_url
            page_dict['title'] = page_title
            page_dict['article date'] = page_date
            page_dict['download date'] = page_dldate
            page_dict['text type'] = page_texttype
            page_dict['text'] = page_text
            page_dict['agency'] = "McKinsey"

            main_dict[page_id] = page_dict
        
# Initiatlize dictionary
main_dict = dict()

#Run spider
process = CrawlerProcess()
siteno = 0
process.crawl(mckinsey_spider)
process.start()

2020-02-18 12:16:09 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2020-02-18 12:16:09 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.6.1, Platform Windows-10-10.0.18362-SP0
2020-02-18 12:16:09 [scrapy.crawler] INFO: Overridden settings: {}
2020-02-18 12:16:09 [scrapy.extensions.telnet] INFO: Telnet Password: e253a11b624c6da5
2020-02-18 12:16:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-02-18 12:16:09 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloade

https://www.mckinsey.com/featured-insights/artificial-intelligence/ai-looks-north-bridging-canadas-corporate-artificial-intelligence-gap


2020-02-18 12:16:10 [urllib3.connectionpool] DEBUG: https://www.mckinsey.com:443 "GET /featured-insights/artificial-intelligence/ai-looks-north-bridging-canadas-corporate-artificial-intelligence-gap HTTP/1.1" 200 19527
2020-02-18 12:16:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.mckinsey.com/featured-insights/artificial-intelligence/the-real-world-potential-and-limitations-of-artificial-intelligence> (referer: None)
2020-02-18 12:16:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.mckinsey.com/featured-insights/artificial-intelligence/tackling-bias-in-artificial-intelligence-and-in-humans> (referer: None)
2020-02-18 12:16:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.mckinsey.com/featured-insights/artificial-intelligence/how-artificial-intelligence-and-data-add-value-to-businesses> (referer: None)
2020-02-18 12:16:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.mckinsey.com/featured-insights/artificial-intelligence/the-ethics

https://www.mckinsey.com/featured-insights/artificial-intelligence/the-real-world-potential-and-limitations-of-artificial-intelligence


2020-02-18 12:16:11 [urllib3.connectionpool] DEBUG: https://www.mckinsey.com:443 "GET /featured-insights/artificial-intelligence/the-real-world-potential-and-limitations-of-artificial-intelligence HTTP/1.1" 200 29503
2020-02-18 12:16:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.mckinsey.com/featured-insights/artificial-intelligence/the-promise-and-challenge-of-the-age-of-artificial-intelligence> (referer: None)
2020-02-18 12:16:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.mckinsey.com/featured-insights/artificial-intelligence/what-southeast-asia-needs-to-become-a-major-player-in-artificial-intelligence> (referer: None)
2020-02-18 12:16:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.mckinsey.com/featured-insights/artificial-intelligence/how-to-ensure-artificial-intelligence-benefits-society-a-conversation-with-stuart-russell-and-james-manyika> (referer: None)
2020-02-18 12:16:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.m

...

2020-02-18 12:28:51 [urllib3.connectionpool] DEBUG: https://www.mckinsey.com:443 "GET /industries/healthcare-systems-and-services/our-insights/the-next-imperatives-for-us-healthcare HTTP/1.1" 200 27479
2020-02-18 12:28:51 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.mckinsey.com:443


https://www.mckinsey.com/featured-insights/asia-pacific/the-asian-century-has-arrived


2020-02-18 12:28:51 [urllib3.connectionpool] DEBUG: https://www.mckinsey.com:443 "GET /featured-insights/asia-pacific/the-asian-century-has-arrived HTTP/1.1" 200 33508
2020-02-18 12:28:51 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.mckinsey.com:443


https://www.mckinsey.com/business-functions/risk/our-insights/transforming-risk-efficiency-and-effectiveness


2020-02-18 12:28:52 [urllib3.connectionpool] DEBUG: https://www.mckinsey.com:443 "GET /business-functions/risk/our-insights/transforming-risk-efficiency-and-effectiveness HTTP/1.1" 200 32788
2020-02-18 12:28:52 [scrapy.core.engine] INFO: Closing spider (finished)
2020-02-18 12:28:52 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 923079,
 'downloader/request_count': 912,
 'downloader/request_method_count/GET': 912,
 'downloader/response_bytes': 18275480,
 'downloader/response_count': 912,
 'downloader/response_status_count/200': 912,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 2, 18, 11, 28, 52, 874330),
 'log_count/DEBUG': 2737,
 'log_count/ERROR': 253,
 'log_count/INFO': 20,
 'response_received_count': 912,
 'scheduler/dequeued': 912,
 'scheduler/dequeued/memory': 912,
 'scheduler/enqueued': 912,
 'scheduler/enqueued/memory': 912,
 'spider_exceptions/IndexError': 253,
 'start_time': datetime.datetime(2020, 2, 18, 11, 16, 9, 

In [123]:
process.stop()

<DeferredList at 0x1de6d0c6cc0 current result: []>

In [124]:
len(main_dict)

578

In [127]:
import json

with open('../data_raw/articles/mckin_articles.json', 'w') as file:
    json.dump(main_dict, file)